# Surface Plasmon Resonance with PyMoosh

In this notebook, we will explore how PyMoosh allows to simulate and understand surface plasmon resonance in the case of a prism coupler. You can do actual research with an apparently so simple program. See [this paper](https://www.dropbox.com/s/r6b2zv2bprzueou/PhysRevA.94.063808.pdf?dl=0). You'll recognize the plots.

First a little background. The surface plasmon is a guided mode along the interface between a metal and a dielectric. Its dispersion relation is analytic, and is given by 
\begin{equation}
 k _x = k_0 \sqrt{\frac{\epsilon_m \epsilon_d}{\epsilon_m + \epsilon_d}}
\end{equation}


## Otto configuration

We first consider the excitation of a surface plasmon using the Otto configuration, which is historically the first one to have been experimentally achieved. 

In [1]:
from PyMoosh import *
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150

wavelength = 600
otto = Structure([1.,'Gold','Water',1.46**2,1.7**2],[4,0,1],[1000,400,200])

List of materials:
Simple, non dispersive: epsilon= 1.0
Database material: Gold, model: Brendel Bormann
Database material: Water, model: Exp Data
Simple, non dispersive: epsilon= 2.1315999999999997
Simple, non dispersive: epsilon= 2.8899999999999997


We can, thanks to the formula above, compute the effective index of the surface plasmon, $n_\text{eff} = \frac{k_x}{k_0}$ and then the corresponding incidence angle $\theta$, using the formula $$ n \sin \theta = n_\text{eff},$$ 

where $n = 1.7$. This means that the evanescent wave generate by the prism will have the same periodicitiy along the interface as the surface plasmon, allowing for it to be excited. 

In [2]:
epsilon_m = otto.materials[1].get_permittivity(600)
n_eff =  np.sqrt(epsilon_m /(1+epsilon_m)) 
print(n_eff)
print(asin(np.real(n_eff)/1.7)/np.pi*180)

(1.0629502930881012+0.011545733508350298j)
38.701631456872285


Now we plot the reflectance of the structure and can see that there is actually a dip in the reflectance right at the computed angle. This is the surface plasmon, which, when excited, leads to a lot of losses in the structure.

In [3]:
wavelength = 600
incidence,r,t,R,T=angular(otto,wavelength,1.,0.,80.,400)
plt.plot(incidence,R)
plt.show()


NameError: name 'angular' is not defined

In [ ]:
window = Window(70*wavelength,0.4,10.,10.)
beam = Beam(wavelength,38.7/180*np.pi,1,10*wavelength)
E=field(otto,beam,window)
plt.figure(2)
plt.imshow(abs(E),cmap='jet',extent=[0,window.width,0,sum(otto.thickness)],aspect='auto')
plt.colorbar()

The surface plasmon does not propagate very far, due to losses. But the resonance at the surface of the metallic plane is obvious. The resonance may shift when the prism is closer to the metallic surface because this perturbs the mode.

## Krestschman Raether configuration

The Krestschmann Raether configuration is obtained by interverting metal and air, essentially. The huge advantage is that the 

In [ ]:
kr = Structure([1.,'Gold','Water',1.46**2,1.7**2],[4,1,0],[500,40,500])
incidence,r,t,R,T=angular(kr,wavelength,1.,0.,80.,400)
plt.plot(incidence,R)
plt.show()


Now it is possible to see that below 35° the gold film being somewhat transparent, the reflection is not that high. Around 35° is the total internal reflection, for $k_x = k_0$. Then, when $k_x = n_\text{eff} k_0$, for a slightly larger incidence angle, a resonance occurs. The surface plasmon is excited through the metal on the metal-air surface. We can have a look at the field map, again.


In [ ]:
window = Window(70*wavelength,0.4,5.,5.)
E=field(kr,beam,window)
plt.figure(2)
plt.imshow(abs(E),cmap='jet',extent=[0,window.width,0,sum(kr.thickness)],aspect='auto')    
plt.colorbar()

We will now include a thin layer of organic material below the metallic film, and put everything in contact with water. This is what occurs when molecules of interest bind themselves to the gold film (usually functionalized) and form a film with a slightly larger refractive index than water (1.46 instead of 1.33 typically). The surface plasmon resonance will then shift. 

In [ ]:
h = np.linspace(0.,20.,8)
for thickness in h :
    kr = Structure([1.,'Gold','Water',1.46**2,1.7**2],[4,1,3,2],[500,40,thickness,500-thickness],verbose = False)
    incidence,r,t,R,T=angular(kr,wavelength,1.,50.,80.,400)
    plt.plot(incidence,R)
plt.show()
